In [1]:
# !pip install tensorflow

In [2]:
# !pip install tensorflow_addons

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, PReLU, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score, 
                             mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error)

from tqdm.auto import tqdm

np.random.seed(12345)
tf.random.set_seed(12345)

/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# ========== Data ==========

In [4]:
data = pd.read_csv('dataset.csv')

In [5]:
data

,x1,x2,y
0,2,1,9
1,2,-1,1
2,-2,2,6
3,1,2,11
4,-2,3,9
5,2,0,5
6,-1,-1,-2
7,-2,1,3
8,0,0,2
9,1,-1,0


In [6]:
X = data[['x1', 'x2']]

In [7]:
y = data['y']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

# ========== Model ==========

In [9]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [10]:
devices_type = [device.device_type for device in tf.config.list_physical_devices()]

devices_type

['CPU']

In [11]:
if 'GPU' in devices_type:
    device_name = '/device:GPU:0'
else:
    device_name = '/device:CPU:0'

In [12]:
best_mse = np.inf

dropout_rates = [0, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for dropout_rate in tqdm(dropout_rates):
    model = Sequential()

    model.add(Dense(5, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
              
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation=None))

    model.compile(loss=MeanSquaredError(), optimizer=SGD(learning_rate=0.01))

    tqdm_callback = tfa.callbacks.TQDMProgressBar(show_epoch_progress=False)

    with tf.device(device_name):
        history = model.fit(X_train, y_train, epochs=500, callbacks=[tqdm_callback], verbose=0)  

    y_pred_val = model.predict(X_val)
    if np.isnan(np.min(y_pred_val)):
        continue
    mse = mean_squared_error(y_val, y_pred_val)
    
    if mse < best_mse:
        best_mse = mse
        best_model = model
        best_dropout_rate = dropout_rate

  0%|          | 0/8 [00:00<?, ?it/s]

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

In [13]:
best_dropout_rate

0.25

In [14]:
best_model.get_weights()

[array([[ 0.365353  ,  0.398407  , -0.01518854, -0.25737092, -0.5124418 ],
        [ 0.5253174 ,  0.5128046 , -1.1286886 , -0.54665786, -1.0990562 ]],
       dtype=float32),
 array([-1.2274706, -1.1868916,  0.8515734,  1.1737835,  1.0092098],
       dtype=float32),
 array([[ 0.44252783, -0.07102477,  0.2722745 ,  0.42174646],
        [ 0.45781714, -0.4294027 ,  0.48525882,  0.32438678],
        [-0.6817098 ,  0.41737354, -0.79922354, -0.56429213],
        [-0.90042037,  0.6453384 , -0.6541083 , -0.90427583],
        [-0.707338  ,  0.8569279 , -0.80257666, -0.8031964 ]],
       dtype=float32),
 array([ 0.95403355, -0.01935067,  1.0183593 ,  1.0458106 ], dtype=float32),
 array([[ 1.4335363],
        [-0.866079 ],
        [ 1.3424886],
        [ 1.4663111]], dtype=float32),
 array([4.423855], dtype=float32)]

# ========== Prediction ==========

In [15]:
X = pd.read_csv('incoming_data.csv')

In [16]:
X

,x1,x2
0,0,3
1,-2,-1
2,0,1
3,0,-1


In [17]:
best_model.predict(X)

array([[9.882976  ],
       [0.13921499],
       [6.166644  ],
       [1.1985273 ]], dtype=float32)